# YOLO Model Training - Indian Food Detection

This notebook trains YOLOv8 on the SOHL Multi-Dish Indian Food Dataset.

## Dataset Info
- **Images**: 377 (301 train, 76 val)
- **Classes**: 16 Indian food categories
- **Format**: YOLO (ready to train)

## Training Parameters
- Model: YOLOv8n (nano - fastest)
- Epochs: 100 (recommended)
- Batch size: 8-16 (adjust based on GPU memory)
- Image size: 640


## Step 1: Install Dependencies


In [ ]:
# Install ultralytics if not already installed
!pip install ultralytics -q


## Step 2: Verify Dataset

**Make sure you've uploaded the `data/yolo_training_data` folder to the server!**


In [ ]:
# Verify dataset exists
import os
from pathlib import Path

dataset_path = Path("data/yolo_training_data/dataset.yaml")

if dataset_path.exists():
    print(f"✅ Dataset found at: {dataset_path}")
    print(f"   Absolute path: {dataset_path.absolute()}")
    
    # Check train/val folders
    train_images = Path("data/yolo_training_data/train/images")
    val_images = Path("data/yolo_training_data/val/images")
    
    if train_images.exists() and val_images.exists():
        train_count = len(list(train_images.glob("*.jpg")) + list(train_images.glob("*.png")))
        val_count = len(list(val_images.glob("*.jpg")) + list(val_images.glob("*.png")))
        print(f"   Train images: {train_count}")
        print(f"   Val images: {val_count}")
    else:
        print("   ⚠️  Train/val folders not found")
else:
    print(f"❌ Dataset not found at: {dataset_path}")
    print("\nPlease upload the dataset first:")
    print("  1. Upload 'data/yolo_training_data' folder to server")
    print("  2. Make sure it contains train/ and val/ folders")


## Step 3: Check GPU Availability


In [ ]:
import torch

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"✅ GPU detected: {torch.cuda.get_device_name(0)}")
    print(f"   GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
    print("\n💡 Training will be much faster on GPU!")
    device = 0  # Use GPU
else:
    print("⚠️  No GPU detected - training will be slower on CPU")
    print("   Consider using a smaller model (yolov8n) or fewer epochs")
    device = "cpu"


## Step 4: Configure Training Parameters


In [ ]:
# Configuration - Adjust these as needed
MODEL_NAME = "yolov8n"  # Options: yolov8n (fastest), yolov8s, yolov8m, yolov8l
DATA_YAML = "data/yolo_training_data/dataset.yaml"
EPOCHS = 100  # Recommended: 100-150
BATCH_SIZE = 16  # Adjust based on GPU memory (8, 16, 32)
IMG_SIZE = 640  # YOLO standard size
OUTPUT_DIR = "models/weights"

print("=" * 60)
print("YOLO Training Configuration")
print("=" * 60)
print(f"Model: {MODEL_NAME}")
print(f"Dataset: {DATA_YAML}")
print(f"Epochs: {EPOCHS}")
print(f"Batch size: {BATCH_SIZE}")
print(f"Image size: {IMG_SIZE}")
print(f"Output: {OUTPUT_DIR}")
print(f"Device: {'GPU' if device != 'cpu' else 'CPU'}")
print("=" * 60)


## Step 5: Load Pretrained Model


In [ ]:
from ultralytics import YOLO

# Load pretrained YOLO model
print(f"\n📥 Loading pretrained {MODEL_NAME}...")
model = YOLO(f"{MODEL_NAME}.pt")
print("✅ Model loaded")


## Step 6: Start Training

**This will take 30-60 minutes on GPU or 3-6 hours on CPU**


In [ ]:
# Start training
print(f"\n🚀 Starting training...")
print(f"Estimated time:")
print(f"  - GPU: 30-60 minutes")
print(f"  - CPU: 3-6 hours")
print("\nTraining in progress...")

results = model.train(
    data=str(Path(DATA_YAML).absolute()),  # Absolute path to dataset.yaml
    epochs=EPOCHS,
    batch=BATCH_SIZE,
    imgsz=IMG_SIZE,
    project=OUTPUT_DIR,
    name="food_detector",
    save=True,
    plots=True,
    val=True,
    device=device  # Use GPU if available
)

print("\n✅ Training complete!")


## Step 7: Check Results and Save Model


In [ ]:
import shutil

# Display results
if results:
    print("\n📊 Training Results:")
    print(f"   mAP50: {results.results_dict.get('metrics/mAP50(B)', 'N/A')}")
    print(f"   mAP50-95: {results.results_dict.get('metrics/mAP50-95(B)', 'N/A')}")
    print(f"   Precision: {results.results_dict.get('metrics/precision(B)', 'N/A')}")
    print(f"   Recall: {results.results_dict.get('metrics/recall(B)', 'N/A')}")

# Find and copy best model
best_model = Path(OUTPUT_DIR) / "food_detector" / "weights" / "best.pt"
final_model = Path(OUTPUT_DIR) / "food_detector_yolo.pt"

if best_model.exists():
    shutil.copy2(best_model, final_model)
    print(f"\n✅ Best model saved at: {final_model}")
    print(f"   File size: {final_model.stat().st_size / 1e6:.1f} MB")
    print(f"\n📥 To download:")
    print(f"   Right-click '{final_model}' in JupyterLab → Download")
    print(f"   Or use: scp user@server:{final_model.absolute()} ./")
else:
    print(f"\n⚠️  Best model not found. Check: {OUTPUT_DIR}/food_detector/weights/")


## Step 8: Test Model (Optional)


In [ ]:
# Test the trained model on a validation image
from PIL import Image
import random

if final_model.exists():
    val_images_dir = Path("data/yolo_training_data/val/images")
    if val_images_dir.exists():
        val_images = list(val_images_dir.glob("*.jpg")) + list(val_images_dir.glob("*.png"))
        if val_images:
            test_image = random.choice(val_images)
            
            # Load trained model
            trained_model = YOLO(str(final_model))
            
            # Run inference
            results = trained_model.predict(str(test_image), conf=0.25)
            
            # Display results
            print(f"Test image: {test_image.name}")
            print(f"Detections: {len(results[0].boxes) if results[0].boxes else 0}")
            
            # Show detected classes
            if results[0].boxes is not None and len(results[0].boxes) > 0:
                for i, box in enumerate(results[0].boxes):
                    class_id = int(box.cls[0])
                    conf = float(box.conf[0])
                    class_name = trained_model.names[class_id]
                    print(f"  {i+1}. {class_name}: {conf*100:.1f}%")
            
            # Display image with predictions
            results[0].show()
        else:
            print("No validation images found")
    else:
        print("Validation directory not found")
else:
    print("Model not found - training may have failed")


## Notes

- **Training time**: 30-60 min (GPU) or 3-6 hours (CPU)
- **Model size**: ~6-12 MB (yolov8n)
- **Expected mAP**: 15-25% (multi-dish detection is challenging)
- **Best model**: Saved automatically during training

## After Training

1. **Download the model** (`food_detector_yolo.pt`)
2. **Place in local project**: `models/weights/food_detector_yolo.pt`
3. **The app will automatically use it!**
